In [ ]:
%%bash

# cd drive/MyDrive/data-files/

# unzip cats_and_dogs.zip

# rm -f cats_and_dogs.zip

# mkdir cats_and_dogs
# mv train cat_and dogs

# ls -l

In [ ]:
import os, shutil

base_dir = 'drive/MyDrive/data-files/cats_and_dogs2'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
validation_dir = os.path.join(base_dir, 'validation')

if os.path.exists(base_dir): # 기존에 디렉터리가 있으면 삭제
  shutil.rmtree(base_dir)

os.mkdir(base_dir)
os.mkdir(train_dir)
os.mkdir(test_dir)
os.mkdir(validation_dir)

train_cat_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cat_dir)
train_dog_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dog_dir)

test_cat_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cat_dir)
test_dog_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dog_dir)

validation_cat_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cat_dir)
validation_dog_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dog_dir)

src_dir = 'drive/MyDrive/data-files/cats_and_dogs/train'
for idx in range(1000):
  fname = 'cat.{0}.jpg'.format(idx)
  shutil.copyfile(os.path.join(src_dir, fname), os.path.join(train_cat_dir, fname))
  fname = 'dog.{0}.jpg'.format(idx)
  shutil.copyfile(os.path.join(src_dir, fname), os.path.join(train_dog_dir, fname))

for idx in range(1000, 1500):
  fname = 'cat.{0}.jpg'.format(idx)
  shutil.copyfile(os.path.join(src_dir, fname), os.path.join(validation_cat_dir, fname))
  fname = 'dog.{0}.jpg'.format(idx)
  shutil.copyfile(os.path.join(src_dir, fname), os.path.join(validation_dog_dir, fname))

for idx in range(1500, 2000):
  fname = 'cat.{0}.jpg'.format(idx)
  shutil.copyfile(os.path.join(src_dir, fname), os.path.join(test_cat_dir, fname))
  fname = 'dog.{0}.jpg'.format(idx)
  shutil.copyfile(os.path.join(src_dir, fname), os.path.join(test_dog_dir, fname))

In [ ]:
import os, shutil

src_dir = 'drive/MyDrive/data-files/cats_and_dogs/train'
base_dir = 'drive/MyDrive/data-files/cats_and_dogs2'
if os.path.exists(base_dir): # 기존에 디렉터리가 있으면 삭제
  shutil.rmtree(base_dir)

dir_names = ['train', 'validation', 'test']
limits = [(0, 1000), (1000, 1500), (1500,2000)]
dir_names2 = ['cats', 'dogs']

os.mkdir(base_dir)
for limits_idx, dir_name in enumerate(dir_names):
  path = os.path.join(base_dir, dir_name)
  os.mkdir(path)
  for dir_name2 in dir_names2:
    path2 = os.path.join(path, dir_name2)
    os.mkdir(path2)
  for idx in range(*limits[limits_idx]):
    for dir_name2 in dir_names2:
      fname = '{0}.{1}.jpg'.format(dir_name2[:3], idx)
      shutil.copyfile(os.path.join(src_dir, fname), os.path.join(path, dir_name2, fname))  

In [ ]:
from tensorflow import keras

In [ ]:
from tensorflow.keras.applications import VGG16

model = VGG16(weights='imagenet',
            include_top=False,
            input_shape=(150, 150, 3))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rescale=1/255) # 특정 디렉터리의 파일을 읽어서 모델의 입력 데이터로 변경하는 도구 (변경 설정 지정 가능)
test_generator = ImageDataGenerator(rescale=1/255)

In [ ]:
X_train = train_generator.flow_from_directory(directory='drive/MyDrive/data-files/cats_and_dogs2/train',
                                              target_size=(150, 150),
                                              batch_size=128,
                                              class_mode="binary") #

X_valid = test_generator.flow_from_directory(directory='drive/MyDrive/data-files/cats_and_dogs2/validation',
                                              target_size=(150, 150),
                                              batch_size=128,
                                              class_mode="binary") #

In [ ]:
history = model.fit_generator(
    X_train,
    steps_per_epoch = 20,
    epochs = 100,
    validation_data = X_valid,
    validation_steps = 20,
)

In [ ]:
# # model save
model.save('dog_cat.h5')

In [ ]:
# 사전학습된 모델의 예측 함수를 호출해서 중간 데이터를 수집
def predict_vgg_values(samples, directory): 
  import numpy as np

  features = np.zeros(shape=(samples, 4, 4, 512))
  labels = np.zeros(shape=(samples,))

  batch_size = 20
  X_train = train_generator.flow_from_directory(directory=directory,
                                                target_size=(150, 150),
                                                batch_size=batch_size,
                                                class_mode="binary")
  for idx, (X, y) in enumerate(X_train):
    print("execute {0} iteration".format(idx + 1))
    predicted_values = vgg.predict(X)
    features[idx*batch_size:(idx + 1)*batch_size] = predicted_values
    labels[idx*batch_size:(idx + 1)*batch_size] = y
    if (idx + 1) * batch_size >= samples:
      break

  return features, labels


In [ ]:
model.save('best_model_val_nonstop.hdf5')

In [ ]:
# train_features, train_labels = predict_vgg_values(2000, 'drive/MyDrive/data-files/cats_and_dogs2/train')
validation_features, validation_labels = predict_vgg_values(2000, 'drive/MyDrive/data-files/cats_and_dogs2/train')

In [ ]:
train_features[0]

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer=keras.optimizers.RMSprop(learning_rate=0.0001),
              metrics=['accuracy'])

In [ ]:
model.fit(train_features, train_labels, batch_size=100, epochs=30, validation_data=(validation_features, validation_labels))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['accuracy'], 'bo', label="accuracy")
plt.plot(history.history['val_accuracy'], 'b', label="val_accuracy")
plt.legend()
plt.show()

In [ ]:
for X_batch, y_batch in X_train:
  print( (X_batch.shape, y_batch.shape) )
  print( y_batch )
  break

In [ ]:
history = model.fit_generator(X_train,
                              steps_per_epoch=100,
                              epochs=30,
                              validation_data=X_valid,
                              validation_steps=50)

In [ ]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['accuracy'], 'bo', label="accuracy")
plt.plot(history.history['val_accuracy'], 'b', label="val_accuracy")
plt.legend()
plt.show()

In [ ]:
# 이미지 증식
train_generator2 = ImageDataGenerator(rescale=1/255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')
test_generator2 = ImageDataGenerator(rescale=1/255,
                                     rotation_range=40,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     shear_range=0.2,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode='nearest')


In [ ]:
X_train2 = train_generator2.flow_from_directory(directory='drive/MyDrive/data-files/cats_and_dogs2/train',
                                                target_size=(150, 150),
                                                batch_size=20,
                                                class_mode="binary") #

X_valid2 = test_generator2.flow_from_directory(directory='drive/MyDrive/data-files/cats_and_dogs2/validation',
                                               target_size=(150, 150),
                                               batch_size=20,
                                               class_mode="binary") #

In [ ]:
from tensorflow.keras.preprocessing import image

idx = 0
for X, y in X_train:
  # print((X.shape, y.shape))
  # print(X[0].shape)
  for idx in range(4):
    plt.imshow(image.array_to_img(X[idx]))
    plt.show()
  break

In [ ]:
# 이미지 크기 150 x 150 x 3

model = keras.Sequential()
model.add(keras.layers.Conv2D(filters=32, 
                              kernel_size=(3, 3),
                              padding="same",
                              activation="relu",
                              input_shape=(150, 150, 3)))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(filters=64, 
                              kernel_size=(3, 3),
                              padding="same",
                              activation="relu"))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(filters=128, 
                              kernel_size=(3, 3),
                              padding="same",
                              activation="relu"))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(filters=128, 
                              kernel_size=(3, 3),
                              padding="same",
                              activation="relu"))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model.compile(loss="binary_crossentropy", 
              optimizer=keras.optimizers.RMSprop(learning_rate=0.0001),
              metrics=['accuracy'])

In [ ]:
history = model.fit_generator(X_train2,
                              steps_per_epoch=100,
                              epochs=30,
                              validation_data=X_valid,
                              validation_steps=50)

In [ ]:
plt.plot(history.history['accuracy'], 'bo', label="accuracy")
plt.plot(history.history['val_accuracy'], 'b', label="val_accuracy")
plt.legend()
plt.show()